## 1. Load and encode datasets with the WeightOfEvidenceEncoder

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../encoders/pattern_preserving/')
from utilities import *

# Load dataset, dropping id column
df = pd.read_csv('train.csv').iloc[0:,1:]
original_cols = df.columns
df.columns = range(len(df.columns))

# Set which variables are categorical
cat_cols = df.columns[:-1]

# Print summary of df
print(df.shape)
df.head()

(300000, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [2]:
# Load encoded dataset if already exists or encode directly
import os.path
import time

if os.path.isfile('encoded_WOE.csv'):
    encoded = pd.read_csv('encoded_WOE.csv', header=None)
else:
    # Import encoder
    from category_encoders import WOEEncoder
    woe = WOEEncoder()
    
    # Introduce categories for numerical features
    temp = set_categories(df[df.columns[:-1]], df.columns[:-1])
    temp[len(temp.columns)] = df[df.columns[-1]]
    df = temp

    # Prepare sample to fit encoder
    X = df[df.columns[:-1]]
    y = df[df.columns[-1]]
    
    # Fit encoder
    tic = time.perf_counter()
    woe.fit(X, y)
    toc = time.perf_counter()
    print('Fitting completed in', round(toc-tic,1), 'seconds')
    
    # Encode whole dataset, and save for future use
    tic = time.perf_counter()
    encoded = woe.transform(df[df.columns[:-1]])
    toc = time.perf_counter()
    print('Transformed full dataset in', round(toc-tic,1), 'seconds')
    encoded[encoded.shape[1]] = df[df.columns[-1]]
    encoded.to_csv('encoded_WOE.csv', index=None, header=None)
    
    # Encode test dataset
    test = pd.read_csv('test.csv', low_memory=False)
    test.columns = range(test.shape[1])
    test = test.loc[0:,1:]    
    test.columns = range(test.shape[1])
    test = set_categories(test, test.columns)
    tic = time.perf_counter()
    test = woe.transform(test)
    toc = time.perf_counter()
    print('Transformed test dataset in', round(toc-tic,1), 'seconds')
    test.to_csv('test_WOE.csv', index=None, header=None)
    
encoded.head()

Fitting completed in 3.5 seconds
Transformed full dataset in 1.0 seconds
Transformed test dataset in 0.8 seconds


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-0.001819,0.111019,-0.020521,-0.015794,-0.075416,0.098327,0.248359,0.006058,-0.317803,-0.345614,...,0.333932,0.132405,0.430146,-0.237516,0.005297,-0.514545,0.420532,0.075086,-0.309031,0
1,-0.001819,-0.354376,-0.020521,-0.015794,-0.075416,0.098327,-0.075660,0.240683,-0.076148,-0.008087,...,-1.052362,-0.134114,0.430146,0.094611,-0.526005,-0.650766,-0.008737,0.157491,0.099991,0
2,-0.001819,0.111019,-0.020521,0.016454,-0.075416,-0.323420,-0.075660,-0.060990,-0.076148,0.217747,...,-0.607677,-0.134114,0.052724,0.426997,0.005297,0.208660,-0.523234,0.157491,-0.309031,0
3,-0.001819,-0.354376,-0.020521,0.016454,-0.075416,0.205037,-0.075660,0.006058,0.155252,0.108884,...,-0.338013,-0.134114,0.430146,0.248265,0.111980,-0.514545,0.227089,0.075086,-0.248787,1
4,-0.001819,0.111019,-0.020521,0.016454,0.128285,0.205037,-0.075660,-0.060990,0.155252,0.108884,...,-0.468414,-0.134114,0.430146,-0.416062,-0.526005,0.208660,0.432324,0.157491,0.099991,0


## 2. Search for best xgboost parameters with Hyperopt

In [3]:
# Define objective function for Hyperopt (credit to Leonardo Ferreira, 
# https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer)
from sklearn.model_selection import KFold
import xgboost as xgb
import time
from hyperopt import hp, fmin, tpe, space_eval

def objective(params):
    tic = time.perf_counter()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n>> Hyperopt New Run")
    print(f"params = {params}")
    FOLDS = 8
    count=1
    max_rounds = 500
    kf = KFold(n_splits=FOLDS, shuffle=False)
    
    mean_auc = 0.0
    for train_idx, val_idx in kf.split(X, y):
        # Prepare data for xgboost
        dtrain = xgb.DMatrix(X.iloc[train_idx, :], label=y.iloc[train_idx])
        dval = xgb.DMatrix(X.iloc[val_idx, :], label=y.iloc[val_idx])
        evallist = [(dtrain, 'train'), (dval, 'eval')]
        
        params['objective']= 'binary:logistic'
        params['eval_metric']= 'auc'
        params['n_thread'] = 7
        evals_result={}
        bst = xgb.train(params, dtrain, max_rounds, evallist, 
                        early_stopping_rounds=30, verbose_eval=False,
                        evals_result=evals_result) 
        
        auc = evals_result['eval']['auc'][-1]
        tauc = evals_result['train']['auc'][-1]
        mean_auc += auc/FOLDS
        print(f'Fold {count} - train_auc {round(tauc, 5)} - val_auc: {round(auc, 5)}')
        count += 1
        del dtrain, dval, bst
    toc = time.perf_counter()
    print('Run time:', round(toc-tic, 1), 'seconds.')
    print('Mean val_auc:', round(mean_auc,5))
    
    return 1-mean_auc

# Define search space for Hyperopt (based on the one by Leonardo Ferreira, 
# https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer)
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),
    'reg_alpha':  hp.uniform('reg_alpha', 0.05, 0.4),
    'reg_lambda': hp.uniform('reg_lambda', 0.05, 0.4),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.15),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),
    'gamma': hp.uniform('gamma', 0.01, 0.7),
    'subsample': hp.choice('subsample', [0.5, 0.6, 0.7, 0.8]),
    'feature_fraction': hp.uniform('feature_fraction', 0.4, 0.8),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 0.9) }

In [4]:
# Take sample for xgboost parameter optimisation
sample = encoded.sample(frac=1, replace=False)
X = sample[sample.columns[:-1]]
y = sample[sample.columns[-1]]

# Run hyperopt 
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=15)

# Set best params
best_params = space_eval(space, best)
best_params 

                                                      
>> Hyperopt New Run
params = {'max_depth': 3, 'gamma': '0.072', 'subsample': '0.80', 'reg_alpha': '0.178', 'reg_lambda': '0.238', 'learning_rate': '0.111', 'colsample_bytree': '0.692', 'feature_fraction': '0.450', 'bagging_fraction': '0.453'}
Fold 1 - train_auc 0.83867 - val_auc: 0.82951         
Fold 2 - train_auc 0.83954 - val_auc: 0.83042         
Fold 3 - train_auc 0.83878 - val_auc: 0.83136         
Fold 4 - train_auc 0.83913 - val_auc: 0.83111         
Fold 5 - train_auc 0.83928 - val_auc: 0.83043         
Fold 6 - train_auc 0.83845 - val_auc: 0.833           
Fold 7 - train_auc 0.83888 - val_auc: 0.83286         
Fold 8 - train_auc 0.83885 - val_auc: 0.83158         
Run time:                                             
239.8                                                 
seconds.                                              
Mean val_auc:                                         
0.83129                                   

{'bagging_fraction': 0.7356582582872502,
 'colsample_bytree': 0.7113276133719563,
 'feature_fraction': 0.6384428432607917,
 'gamma': 0.26026482549588614,
 'learning_rate': 0.06958209795561189,
 'max_depth': 3,
 'reg_alpha': 0.34189321752780444,
 'reg_lambda': 0.07072864459293252,
 'subsample': 0.6}

## Train best model further and make predictions to submit

In [5]:
# Train xgboost model further with best params and full dataset
from sklearn.model_selection import train_test_split
train, val = train_test_split(encoded, train_size=0.75, random_state=0)
dtrain = xgb.DMatrix(train[train.columns[:-1]], label=train[train.columns[-1]])
dval = xgb.DMatrix(val[val.columns[:-1]], label=val[val.columns[-1]])

evallist = [(dtrain, 'train'), (dval, 'eval')]
evals_result = {}
best_params['objective']= 'binary:logistic'
best_params['eval_metric']= 'auc'
best_params['n_thread'] = 7
bst = xgb.train(best_params, dtrain, 4000, evallist, early_stopping_rounds=50, 
                verbose_eval=False, evals_result=evals_result) 

print('Train auc '+str(evals_result['train']['auc'][-1])+', Test auc '+str(evals_result['eval']['auc'][-1]))

Train auc 0.839698, Test auc 0.830302


In [6]:
# Load test set
test = pd.read_csv('test_WOE.csv', header=None, low_memory=False)
index = pd.read_csv('test.csv', low_memory=False)['id'].values
dtest = xgb.DMatrix(test)
# Make predictions and save
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

if not os.path.exists('submissions'):
    os.mkdir('submissions')
pd.DataFrame({'id':index, 'target':ypred}, index=None).to_csv('./submissions/predicted_WOE.csv', index=None)